In [66]:
from torch.utils.data import Dataset
import random
import torch
import os
from torch.utils.data import DataLoader

import pandas as pd

import numpy as np

class ClfDataset(Dataset):

    def __init__(self, train=True):
        self.train = train
        data_dir = './dataset/'
        # choose the dataset
        patients_train = os.listdir(data_dir+'train_val/')
        patients_test = os.listdir(data_dir+'test/')
        labels_df = pd.read_csv('./dataset/info.csv',index_col=0)

        self.data_train = []
        self.data_test = []
        self.label = []

        for num, patient in enumerate(patients_train):
            patient_name = patient[0:-4]
            label = labels_df.get_value(patient_name, 'lable')
            path = data_dir + 'train_val/' + patient
            img_data = np.load(path)
            voxel = img_data['voxel'].astype(np.int32)
            self.data_train.append(voxel)
            self.label.append(label)

        for num, patient in enumerate(patients_test):
            path = data_dir + 'test/' + patient
            img_data = np.load(path)
            voxel = img_data['voxel'].astype(np.int32)
            self.data_test.append(voxel)
    
    def __getitem__(self, item):
        if self.train:
            patient_data = self.data_train[item]
            patient_label = self.label[item]
            return patient_data, patient_label
        else:
            patient_data = self.data_test[item]
            return patient_data
        
    def __len__(self):
        return len(self.label)

In [67]:
data_ineed = ClfDataset(train=True)
train_data_loader = DataLoader(dataset=data_ineed, batch_size=32, shuffle=True)



/home/vivi/anaconda3/envs/3dunet/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [68]:
data_ineed[100][0]

array([[[164, 176, 185, ..., 176, 172, 170],
        [172, 181, 172, ..., 176, 174, 170],
        [176, 170, 163, ..., 178, 178, 185],
        ...,
        [191, 255, 255, ..., 215, 186, 184],
        [185, 255, 255, ..., 220, 186, 185],
        [192, 220, 255, ..., 222, 191, 199]],

       [[163, 168, 178, ..., 175, 174, 171],
        [166, 175, 173, ..., 181, 178, 172],
        [172, 171, 167, ..., 182, 177, 183],
        ...,
        [181, 255, 255, ..., 216, 181, 181],
        [178, 215, 255, ..., 225, 181, 185],
        [183, 190, 255, ..., 222, 183, 193]],

       [[161, 163, 170, ..., 174, 176, 170],
        [164, 171, 171, ..., 177, 176, 174],
        [169, 171, 169, ..., 178, 176, 179],
        ...,
        [189, 244, 255, ..., 214, 177, 182],
        [187, 204, 255, ..., 229, 181, 183],
        [191, 183, 255, ..., 222, 184, 194]],

       ...,

       [[  5,   2,   3, ...,   6,   5,   1],
        [  4,   1,   4, ...,   6,   4,   4],
        [  2,   2,   5, ...,   6,   5,   3